# Tree Methods

1. Regression Trees
2. Classification Trees
3. Trees vs. Linear Models

The lecture draws from Chapter 8 of James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). "An introduction to statistical learning: with applications in r."

---
# 1. Regression Trees

Up until this point in class we have largely focused on parametric models for regression questions (e.g., linear regression, LASSO, PCR). The only exception to this was our discussion of _kNN Regression_. In these next two lectures we will introduce a set of powerful non-parametric methods for regression and classification questions: _tree methods_.

Unlike standard linear models, like ordinary least squares regression, _tree methods_ try to identify sub-regions in your $X$ data space that explain variability in $Y$. You can think of this as identifying a specific set of rules for identifying _local means_ in $Y$. These rules take the form of simple _if/then_ statements:

"**If** the values in $X_1, ..., X_p$ have this specific set of properties, **then** $y$ is a particular value."



### Goal: 

***Your goal with learning a regression tree is to identify the best set of decision rules for dealing with each feature in $X$ that maximally explain variability in $Y$.***

The if/then structure of the logic of regression trees is designed to partition your predictor variable space $X$ into a set of _regions_ ($R$). In the regression context, each region $R_i$ is associated with a specific mean of $Y$. Thus, regression tree models try to find the fewest number of regions that explain the most variance in the response variable. 


## Recursive binary splitting


Constructing a regression tree involves two steps:

1. _Divide $X$_: Break the set of all possible values for $X_1, ..., X_p$ into _j_ distinct subregions, $R_1, ..., R_j$.

2. _Assign $Y$_: For every observation within a subregion $R_j$, assign the same value for $Y$ (i.e., $\hat{y}_j$). 

The book shows an example of this idea for the baseball dataset. Here the regression tree makes a prediction of player salary based on total number of hits in the year ($Hits$) and experience ($Years$). 

![RegressionRegions](imgs/L20BaseballRegions.png)

Notice that the joint distribution of $Hits$ and $Years$ is broken into three subspaces: $R_1, R_2, R_3$. Let's explore each subspace in turn.

The region $R_1$ is defined as any player with less than 4.5 years of experience. If a player falls into this category, regardless of their $Hits$ value, then their predicted salary is $\hat{y}=\$165,174$ (Note: in the book & figures, the values are on a log-scale, so $R_1 = 5.107$ is really $R_1 = \$1000 x e^{5.107}$)

Now for players with more than 4.5 years of experience, the regression tree further separates them into two regions $R_2$ \& $R_3$, for players with $Hits$ values of _less_ than or _more_ than 117.5 respectively. For players who fall into $R_2$ (i.e., $Years > 4.5$ \& $Hits<117.5$), we predict their salary to be $\$402,834$. For players who fall into $R_3$ (i.e., $Years > 4.5$ \& $Hits>117.5$), we predict their salary to be $\$845,346$.

Notice how $R_1$ does not have any sub-splits along the $Hits$ dimension. This is because tree methods work in a ***recursive*** manner. Technically the term is called _recursive binary splitting_ and it works in this way. Define the one variable to _split_ such that you account for the most variance in $Y$, then find ways of further splitting along the other variables. In this way, later subregions $R_{j+1}, R_{j+2}, ...$ are defined as splits from the current subregion $R_j$. 

The process of recursive binary splitting is why we call this process a _tree_ method. You can visualize the decision rules for defining all regions $R_1, ..., R_j$ using a dendogram and this takes on a tree-like structure.

In the baseball example, we can visualize the tree like this.

![RegressionTree](imgs/L20BaseballTree.png)

<br>

---
### Terms:

* **Terminal Nodes (aka- Leaves):** The regions $R_1, ..., R_j$.
* **Internal Nodes:** The points where $X$ is split.
* **Branches:** Segments of the tree that connect nodes.

The terminal nodes are the nodes that you see at the end (bottom) of the tree. The path to get there involves following downwards along each of the internal nodes that define the splits.

---

<br>


As I mentioned at the beginning, tree methods are non-parametric (i.e., they don't assume a particularly form of the distribution of $Y$). However, tree methods are still supervised. In particular, the split points for all regions $R_1, ..., R_j$ are chosen by minimizing the following definition of the residual square error (RSS).

$$ \sum_{j=1}^J \sum_{i \in R_j} (y_i - \hat{y}_{R_j})^2 $$

In this case, $\hat{y}_{R_j}$ is the mean value for the training observations within the $j^{th}$ region. 

The recursive binary splitting algorithm is the most computationally efficient algorithm for finding the best way to minimize the objective function above because comparing _all possible split points_ is just too computationally expensive and would make a tree fairly useless as a method. We say that the approach is _top-down_ and _greedy_.

* **Top-down:** It begins at the top of the tree, splitting along one feature.
* **Greedy:** At each step, the "best" split is chosen just for that step.

The top-down part should be pretty intuitive. Let's explore the greediness a bit more. For each region, we select the cutpoint $s$ that leads to the greatest possible reduction in RSS as defined above. For a given predictor $X_j$, we define the regions of that predictor now as:

$$R_1(j,s) = \{X|X_j < s\} \text{  and  } R_2(j,s)= \{X|X_j \geq s\}$$

Here $X$ is the preceding subspace of the predictor variable (all of $X$ if $X_j=X_1$).

Thus the specific value of $s$ chosen is the one that mimizes this equation:

$$\sum_{i: x_i \in R_1(j,s)} (y_i - \hat{y}_{R_1})^2  + \sum_{i: x_i \in R_2(j,s)} (y_i - \hat{y}_{R_2})^2$$



## Bias-Variance Tradeoff

The tree generation procedure described thusfar has no limit on the number of splits. As the number of splits increases (i.e., the tree gets "bigger"), then the model's flexibility and variance also increase. In this way, we define size of the tree (number of splits and regions) as determining the model flexibility.

This means that _parsimony_ in regression trees is determined by finding the smallest possible tree that explains the most variance in $Y$.

The most intuitive method for identifying the most parsimonious tree for your data is to start with the most complex tree that you can ($T_0$) and fine a way to **prune** it back (i.e., remove unnecessary splits). This method is known as **cost-complexity pruning** or **weakest-link pruning** and it is implemented using a sparsity constraint much like we learned about for LASSO and ridge regression.

We now need to define a new term $\alpha$ (this is not the same $\alpha$ that we learned for elastic net). Here $\alpha$ is a non-negative tuning parameter that is used to reduce the number of terminal nodes (i.e., regions) $T$ that a fit procedure will return. $\alpha$ thus modifies the objective function to:

$$ \sum_{m=1}^{|T|} \sum_{i: x_i \in R_m} (y_i - \hat{y}_{R_m})^2 + \alpha |T| $$

As $\alpha$ gets larger, it makes it harder for the fit to return trees with a large number of temrinal nodes. If $\alpha=0$ then the most complex tree ($T_0$) is selected. So $\alpha$ controls the size of your tree $T$ that, in turn, defines the bias-variance tradeoff. This tradeoff is illustrated for the baseball example in the figure below.

![TreeVarianceBias](imgs/L20TreeBiasVariance.png)

Here the test error for k-fold cross validation (green) is compared against a split-half test set (brown). Both show a similar dip in the residual error for trees with $T=3$.


---
# 2. Classification Trees

At this point it should seem pretty intuitive how tree methods can be converted from a regression context to a classification context. Rather than return the mean value of $Y_j$ at each region $R_j$, _classification trees_ return the modal (or commonly occuring) category of $Y$. 

The only thing you have to do to convert a regression tree into a classification tree is modify your error term. Now at first glance, a reasonal estimation of the classification error for the $m^{th}$ region of your predictor variable space would be

$$ E_m = 1 - \max_{k}(\hat{p}_{mk}) $$

Where $\hat{p}_{mk}$ is the proportion of training observations in the $m^{th}$ region that are members of the $k^{th}$ class (or category). Thus, for $E_m$ you are just taking your error from the _best_ represented class or category.

The problem with this intuitive approach to estimating error for fitting a classification tree is that it is not sensitive _enough_.

There are, instead, two alternative definitions of error that allow for enough sensitivity for finding optimal classification trees:

<br>

1) **Gini index:** This is a measure of total variance across all class/categories.

$$ G = \sum_{k=1}^{K} \hat{p}_{mk}(1-\hat{p}_{mk}) $$

The Gini index is a measure of _node purity_, in that a small $G$ indicates that a node contains predominantly members of a single class or category. Larger $G$ indicates that each node has a greater mixture of classes or categories.

<br>

2) **Cross-entropy:** A similar measure of node purity, defined slightly differently.

$$ D = - \sum_{k=1}^{K} \hat{p}_{mk} \log  \hat{p}_{mk} $$

In this case, $D$ will be near zero if all of the $\hat{p}_{mk}$'s are near zero or near one. This means that $D$ \& $G$ produce numerically similar outcomes (i.e., similar fit results). 

An example of a classification tree is shown for the $Heart$ dataset in the book. See Chapter 8 for details. For now pay attention to all of the terminal nodes.

![ClassificationTree](imgs/L20ClassificationTree.png)

One thing to pay attention to is that some of the splits result in terminal nodes that have the exact same predicted value (e.g., $RestECG<$). This is the result of using the node purity measures. Even though these splits produce redundant outcomes, they also increae the resulting node purity. For example, one side of the split might have 100\% training set observations from Category 1 (Yes), while the other side of the split might have 75\% be Category 1. The majority vote is the same within each region, but you have maximized the purity of the respective nodes.



---

# 3. Trees vs. Linear Models

When to use tree methods, as opposed to linear parametric methods, depends a lot on the goals of the analysis and the nature of the data set. Tree methods work better in some contexts than others. 

In general, tree methods work well when there is a highly non-linear and complex relationship between $X$ and $Y$. It is probably easier to see this in the classification context. The figure below shows two contexts for a categorization problem where $Y$ is binary, and $X$ has two features. Here the green regions show the case where $Y=0$ and the yellow regions show where $Y=1$. The top row is a case where $Y$ is linearly separable according to $X1$ and $X2$. The bottom row shows a case where $Y$ is **not** linearly separable according to $X1$ and $X2$. The left column shows how a linear model (i.e., logistic regression) would solve this. The right column shows how a classification tree would solve the problem.

![TreeVsLinearModel](imgs/L20TreeVsLinearModel.png)

Notice that the linear model does a good job of finding the category boundary when the data fit the linearity assumptions (top left). In contrast, the classification tree idetnifies a lot of regions with high classification error (top right). The opposite is true, however, when the data do not fit the linearity assumptions. The category line from the linear model produces two classification regions with a lot of error (bottom left), where has the classficiation tree finds three regions with perfect node purity.

Beyond this ability to describe non-linear relationships, tree methods have other advantages over linear models.

<br>

### Advantages of trees

* **Trees are easy to explain:** The rules for the different splits are easy to verbalize.
* **Closely mirror human decision-making:** This is an advantage that the book specifies, but as a cognitive scientist I strongly disagree with.
* **Trees can easily be displayed graphically:** This makes them easy to interpret.
* **No need for dummy variables:** You don't need to transform categorical predictor variables in the same way you would for linear or logistic regression.

<br>

Of course, tree methods aren't perfect.

<br>

### Disadvantages of trees

* **Lower predictive accuracy:** At least compared to linear models.
* **Non-robust:** A small change in the data can cause a massive change in the nature of the estimated tree.

<br>

We will follow up on these limitations in the next lecture.